<h1><center>Minimal Covers</center></h1>

Devise and implement three Python functions that compute a minimal cover, all accessible (by the algorithm given in the lecture) minimal covers, and all possible minimal covers of a given list of functional dependencies, respectively.
Complete the provided Jupyter Notebook using the Python 3 kernel.
Do not change the name and the parameters of the functions in the template.
You may add ancillary functions if necessary or convenient.
You may import additional Python standard libraries but no external libraries.

For all questions, use the following data structures.
A functional dependency is represented as a list of two lists. For
example, the functional dependency {A, B} → {C} is represented as [['A', 'B'], ['C']].
A set of functional dependencies is represented as a list of functional dependencies.

Ensure that your functions process the input and produce the output in the format specified in the corresponding questions.

For simplicity, you may assume that the input is always valid (e.g. no empty string, no incomplete functional dependency).
Consider the case in which a functional dependency's left- or right-hand side is empty.

The code should be readable and adequately commented.

The marking team considers clarity of the code and comments, quality of the algorithm and  implementation, and correctness and efficiency of the code.

### Indicate your student number: A0276571W

# Question 1 (4 points)
Write a function *min_cover* that takes a list of functional dependencies as input and returns a minimal cover of the functional dependencies, for example,
````
min_cover([[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]])
outputs: [[['A'], ['B']], [['B'], ['C']], [['B'], ['D']], [['D'], ['B']], [['A', 'E'], ['F']]]
````
The results is a minimal cover, i.e. a list of functional dependencies.
There might be more than one correct answer to this question.

In [1]:
def get_closure(fds, attributes):
  fds_copy = fds.copy()
  closure_result = attributes.copy()
  stop = False
  while not stop:
    stop = True
    remove_list = []
    # traverse the fds and find if we can make new progress
    for fd in fds_copy:
      # make new progress and keep searching
      if all(left_element in closure_result for left_element in fd[0]):
        stop = False
        closure_result = list(set(closure_result + fd[1]))
        remove_list.append(fd)
    # if no new progress, quit the traversing
    for i in remove_list:
      fds_copy.remove(i)
  closure_result.sort()
  return closure_result


def min_cover(fds):
    # Your code here:
    # step1: simplify the right hand side
    right_hand_side = []
    for fd in fds:
      if len(fd[1]) > 1:
        for i in fd[1]:
          right_hand_side.append([fd[0], [i]])
      else:
        right_hand_side.append(fd)
    # print(right_hand_side)
    # step2: simplify the left hand side
    left_hand_side = []
    for fd in right_hand_side:
      if len(fd[0]) > 1:
        left = fd[0].copy()
        stop = False
        # need to subtract itself
        right_hand_side_copy = right_hand_side.copy()
        right_hand_side_copy.remove(fd)
        # keep simplifying until no progress or left hand is simplified
        while len(left) > 1 and not stop:
          stop = True
          remove_list = []
          # iterate every element, find all elements that can be removed:
          for element in left:
            # subtract itself, and judge whether it can be removed
            left_copy = left.copy()
            left_copy.remove(element)
            # print(right_hand_side_copy)
            new_closure = get_closure(right_hand_side_copy, left_copy)
            '''
            if we can still get element after we remove it
            or right hand side can still be derived
            then we can remove the element and keep searching
            '''
            if (element in new_closure) or (fd[1] in new_closure):
              remove_list.append(element)
              stop = False
          for i in remove_list:
            if len(left) > 1:
              left.remove(i)
        # finish simplifying
        left_hand_side.append([left, fd[1]])
        # print(left_hand_side)
      # no need to simplify
      else:
        left_hand_side.append(fd)
    # print(left_hand_side)
    # step3: simplify the set itself
    # only need to delete from step2
    minimal_cover = left_hand_side.copy()
    stop = False
    while not stop:
      stop = True
      remove_list = []
      # traverse every fd
      for fd in minimal_cover:
        deleted = minimal_cover.copy()
        deleted.remove(fd)
        # print(deleted)
        deleted_closure = get_closure(deleted, fd[0])
        # print(deleted_closure)
        # if fd[1][0] can be derived from other fds, then it's redundant
        if fd[1][0] in deleted_closure:
          # remove_list.append(fd)
          minimal_cover.remove(fd)
          stop = False
      # for i in remove_list:
      #   minimal_cover.remove(i)
    # finish step3 and return

    return minimal_cover

### Expected Result

In [2]:
fds = [[['A'], ['B', 'C']],[['B'], ['C','D']], [['D'], ['B']], [['A','B','E'], ['F']]]

min_cover(fds)

[[['A'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['D'], ['B']],
 [['A', 'E'], ['F']]]

**The result you get should be similar to:**
```
[[['A'], ['B']],
 [['B'], ['C']],
 [['B'], ['D']],
 [['D'], ['B']],
 [['A', 'E'], ['F']]]
```

# Question 2 (3 points)
Write a function *min_covers* that takes a list of functional dependencies as input and returns all minimal covers reachable from the given list of functional dependencies, for example,
````
min_cover([[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]])
outputs: [
            [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
            [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
        ]
````
The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.

In [3]:
import itertools

def check_duplicate_fds(fds):
  # input a fds(cover), remove duplicate fds
  fds.sort()
  remove_list = []
  for i in range(len(fds) - 1):
    if fds[i] == fds[i + 1]:
      remove_list.append(i + 1)
  remove_list.sort(reverse=True)
  for i in remove_list:
    del fds[i]
  return fds

def check_duplicate_covers(covers):
  # input a list of covers, remove duplicate fds and then covers
  for cover in covers:
    cover = check_duplicate_fds(cover)
  covers.sort()
  remove_list = []
  for i in range(len(covers) - 1):
    if covers[i] == covers[i + 1]:
      remove_list.append(i + 1)
  remove_list.sort(reverse=True)
  for i in remove_list:
    del covers[i]
  return covers

def min_covers(fds):
    # Your code here
    # step1: simplify the right hand side
    # nothing different from min_cover()
    right_hand_side = []
    for fd in fds:
      if len(fd[1]) > 1:
        for i in fd[1]:
          right_hand_side.append([fd[0], [i]])
      else:
        right_hand_side.append(fd)
    # print(right_hand_side)
    # step2: simplify the left hand side
    '''
    left_hand_side is used to store all possible fds generated by all fds one by one
    if len(fd[0]) == 1, the list will only be [fd]
    else, the list will be [[possible1 -> fd[1]], [possible2 -> fd[1]], ...]
    '''
    left_hand_side = []
    for fd in right_hand_side:
      if len(fd[0]) > 1:
        left = fd[0].copy()
        # need to subtract itself
        right_hand_side_copy = right_hand_side.copy()
        right_hand_side_copy.remove(fd)
        '''
        for each len(left_hand_side) > 1, we need to get all possible permutation and combination
        use itertools.permutation()
        all_left is used to store all possible fds generated by this one fd
        for example, if ['A','B'] -> ['C'] need to be simplifies, then it will contain both ['A'] -> ['C'] and ['B'] -> ['C']
        '''
        all_left = []
        all_possible = list(itertools.permutations(left))
        for every_left in all_possible:
          every_left = list(every_left)
          stop = False
          # keep simplifying until no progress or left hand is simplified
          while len(every_left) > 1 and not stop:
            stop = True
            remove_list = []
            # iterate every element, find all elements that can be removed:
            for element in every_left:
              # subtract itself, and judge whether it can be removed
              every_left_copy = every_left.copy()
              every_left_copy.remove(element)
              # print(right_hand_side_copy)
              new_closure = get_closure(right_hand_side_copy, every_left_copy)
              '''
              if we can still get element after we remove it
              or right hand side can still be derived
              then we can remove the element and keep searching
              '''
              if (element in new_closure) or (fd[1] in new_closure):
                remove_list.append(element)
                stop = False
            for i in remove_list:
              if len(every_left) > 1:
                every_left.remove(i)
          # finish simplifying, then add this fd into all_left
          all_left.append([every_left, fd[1]])
        # after collecting all_left, check order and duplicate
        for iter in all_left:
          iter[0].sort()
        all_left = check_duplicate_fds(all_left)
        left_hand_side.append(all_left)
      # no need to simplify, add whole [fd]
      else:
        left_hand_side.append([fd])
    # print(left_hand_side)
    # final_left_hand_side is used to store all combination combined from left_hand_side
    final_left_hand_side = []
    for covers in left_hand_side:
      # initiation
      if len(final_left_hand_side) == 0:
        for fd in covers:
          final_left_hand_side.append([fd])
      # combination
      else:
        temp = []
        for fd in covers:
          for combination in final_left_hand_side:
            temp.append(combination + [fd])
        final_left_hand_side = temp
    # print(final_left_hand_side)
    # step3: simplify the set itself
    # only need to delete from step2
    minimal_covers = []
    # deal with each possible combination
    for every_left_hand_side in final_left_hand_side:
      # like step2, if we want to get all possible reachable covers, we need to first get all possible combinations
      all_possible_left_hand_side = list(itertools.permutations(every_left_hand_side))
      # for each possible combination, simplify it by step3
      for cover in all_possible_left_hand_side:
        cover = list(cover)
        stop = False
        while not stop:
          stop = True
          remove_list = []
          # traverse every fd
          for fd in cover:
            deleted = cover.copy()
            deleted.remove(fd)
            deleted_closure = get_closure(deleted, fd[0])
            # if fd[1][0] can be derived from other fds, then it's redundant
            if fd[1][0] in deleted_closure:
              cover.remove(fd)
              stop = False
        minimal_covers.append(cover)
    # finish step3 and return
    minimal_covers = check_duplicate_covers(minimal_covers)

    return minimal_covers

### Expected Result

In [4]:
fds = [[['A'], ['A', 'B']], [['B'], ['A','C']], [['A'], ['C']], [['A','B'], ['C']]]
min_covers(fds)

[[[['A'], ['B']], [['A'], ['C']], [['B'], ['A']]],
 [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]]]

**The result you get should be similar to:**
```
[
    [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']]],
    [[['A'], ['B']], [['B'], ['A']], [['A'], ['C']]]
]
```

# Question 3 (3 points)
* Write a function *all_min_covers* that takes a list of functional dependencies as input and returns all minimal covers, for example,
````
all_min_covers([[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]])
outputs: [
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````

The results is a list of minimal covers.
Make sure that you clearly explain the rationale of your solution in comments of the code.
This question is challenging and not everyone may find a solution.

In [5]:
def get_subsets(relations):
    subsets = []
    for r in range(1, len(relations) + 1):
      subsets.extend(itertools.combinations(relations, r))
    result = []
    for i in subsets:
      i = list(i)
      result.append(i)
    return result

def get_all_closures(relations, fds):
    # given the relations and fds, return all closures
    result = []
    candidates = []
    subsets = get_subsets(relations)
    for subset in subsets:
      if not any(all(i in subset for i in candidate) for candidate in candidates):
        closure = get_closure(fds, subset)
        result.append([subset, closure])
        if len(closure) == len(relations):
          candidates.append(subset)
    result.sort(key=lambda x: len(x[1]), reverse=True)
    return result

def get_all_fds(relations):
    # given the relations, create each possible fd combination
    # but for simplicity, the right hand side will only contain 1 element
    all_fds = []
    for right_hand_side in relations:
        relations_copy = relations.copy()
        relations_copy.remove(right_hand_side)
        # all possible combinations of relations excluding itself
        all_left_hand_side = get_subsets(relations_copy)
        for left_hand_side in all_left_hand_side:
            all_fds.append([left_hand_side, [right_hand_side]])
    return all_fds

def all_minimal_covers(fds):
    # your code here
    # first we need to get the relations, which is the list of all attributes appeared at least once
    relations = []
    for fd in fds:
      for element in fd[0]:
        relations.append(element)
      for element in fd[1]:
        relations.append(element)
    relations = list(set(relations))
    relations.sort()
    '''
    then we need to create all the possible fds according to relations we get
    each subset of all the fds can be regarded as a fds condition given to us in Question 1 and 2
    '''
    all_possible_fds = get_all_fds(relations)
    all_possible_fds_conditions = get_subsets(all_possible_fds)
    given_closures = get_all_closures(relations, fds)
    given_closures.sort()
    '''
    ofc, we need to filter all_possible_fds_conditions because not every fds_condition is equivalent to the given fds
    so we need to compare their closures, and all the equivalent fds conditions will be stored in final_fds_conditions
    '''
    final_fds_conditions = []
    for fds in all_possible_fds_conditions:
      #in order to compare the validity, need to call get_all_closures with each specific fds
      specific_fds_closures = get_all_closures(relations, fds)
      # first compare the length, then the content
      if len(specific_fds_closures) == len(given_closures):
        specific_fds_closures.sort()
        if specific_fds_closures == given_closures:
          final_fds_conditions.append(fds)
    # after appending all valid combination of fd, append itself
    final_fds_conditions.append(fds)
    check_duplicate_fds(final_fds_conditions)
    all_minimal_covers = []
    count = 1
    # then we only need to call min_covers() of Question2, and eliminate all duplicates
    for fds in final_fds_conditions:
      all_min_covers = min_covers(fds)
      for cover in all_min_covers:
        all_minimal_covers.append(cover)
      print(count)
      count += 1
    # handle duplication
    check_duplicate_covers(all_minimal_covers)

    return all_minimal_covers

### Expected Result

In [6]:
fds = [[['A'], ['B']],[['B'], ['C']], [['C'], ['A']]]
all_minimal_covers(fds)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


[[[['A'], ['B']], [['A'], ['C']], [['B'], ['A']], [['C'], ['A']]],
 [[['A'], ['B']], [['B'], ['A']], [['B'], ['C']], [['C'], ['B']]],
 [[['A'], ['B']], [['B'], ['C']], [['C'], ['A']]],
 [[['A'], ['C']], [['B'], ['A']], [['C'], ['B']]],
 [[['A'], ['C']], [['B'], ['C']], [['C'], ['A']], [['C'], ['B']]]]

**The result you get should be:**
```
[
	[[['A'], ['C']], [['B'], ['A']], [['C'], ['A']], [['A'], ['B']]],
	[[['B'], ['C']], [['C'], ['A']], [['A'], ['B']]],
	[[['C'], ['B']], [['A'], ['C']], [['C'], ['A']], [['B'], ['C']]],
	[[['C'], ['B']], [['A'], ['C']], [['B'], ['A']]],
	[[['B'], ['C']], [['A'], ['B']], [['B'], ['A']], [['C'], ['B']]]
]
````